<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Splitting%20Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Text Splitting**
---
El **Text Splitting** (o división de texto) es una etapa crítica en los sistemas de **Retrieval-Augmented Generation (RAG)**. Consiste en fragmentar documentos extensos en segmentos más pequeños y manejables (_chunks_) antes de indexarlos para su recuperación posterior. Su implementación incide directamente en la calidad de la recuperación de información y, por ende, en las respuestas generadas por el modelo.  

---

### **¿Por qué es importante?**

1.  **Equilibrio entre contexto y precisión**:
    
    -   **Chunks demasiado grandes**: Pueden contener información redundante o irrelevante, lo que "diluye" el contexto clave y reduce la precisión del retrieval.
        
    -   **Chunks demasiado pequeños**: Pierden contexto necesario para entender el significado completo (p. ej., una frase sin su párrafo asociado).
        
    -   Un buen _splitting_ mantiene la coherencia semántica en cada segmento, facilitando que el sistema recupere los fragmentos más relevantes para una consulta.
        
2.  **Compatibilidad con modelos de embedding**:
    
    -   Los modelos de embedding (como SBERT o OpenAI) tienen límites óptimos de longitud de texto. Por ejemplo, un chunk de 512 tokens funciona bien con muchos codificadores, pero un texto más largo podría truncarse o perder información crítica.
        
3.  **Impacto en la generación de respuestas**:
    
    -   Los chunks recuperados alimentan al modelo generador (como GPT-4). Si están mal estructurados, el modelo recibirá información fragmentada o fuera de contexto, lo que generará respuestas inconsistentes o inexactas.
        
4.  **Eficiencia computacional**:
    
    -   Chunks bien dimensionados reducen costos de procesamiento y latencia, ya que evitan sobrecargar el sistema con datos innecesarios.


### El Text Splitting no es un paso mecánico, sino una decisión estratégica que determina cómo el sistema "ve" la información y afectara enormemente a la calidad del sistema.

# **Nivel 1: División por Caracteres**  
---
La **división por caracteres** es la forma más básica de fragmentar texto. Consiste simplemente en dividir el texto en _chunks_ (segmentos) de un tamaño fijo de **N caracteres**, ignorando por completo el contenido o estructura del texto.

Este método **no se recomienda para aplicaciones reales**, pero es un punto de partida útil para comprender los fundamentos del _text splitting_.

**Ventajas**:

-   Fácil y sencillo de implementar.
    

**Desventajas**:

-   Muy rígido: **no considera la estructura del texto** (p. ej., separación de párrafos, puntuación o temas).
    
-   Puede fragmentar ideas o contextos clave a la mitad.
    

**Conceptos clave**:

-   **Tamaño del chunk (_Chunk Size_)**: Número de caracteres por segmento (ej: 50, 100, 1000).
    
-   **Solapamiento de chunks (_Chunk Overlap_)**: Cantidad de caracteres que se superponen entre chunks consecutivos. Esto ayuda a evitar que una misma información contextual quede dividida en múltiples chunks, aunque genera duplicación de datos.



In [ ]:
text = "El text splitting divide textos en segmentos pequeños para facilitar su procesamiento. Es clave mantener el equilibrio entre tamaño y contexto. Un buen splitting mejora la precisión en sistemas como RAG."

In [ ]:
# Creamos una lista para almacenar los chunks
chunks = []

chunk_size = 35 # Caracteres

# desde 0 hasta la longitud del texto con salto de chunk_size
for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
chunks

['El text splitting divide textos en ',
 'segmentos pequeños para facilitar s',
 'u procesamiento. Es clave mantener ',
 'el equilibrio entre tamaño y contex',
 'to. Un buen splitting mejora la pre',
 'cisión en sistemas como RAG.']

Langchain tiene una clae que permite realizar esta tarea facilmente

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# Instanciamos un objeto CharacterTextSplitter y le pasamos los pasarmetros de corte
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=0, separator='', )

# Then we can actually split our text via create_documents.
# Note: create_documents expects a list of texts, so if you just have a string (like we do) you'll need to wrap it in []
text_splitter.create_documents([text])


[Document(metadata={}, page_content='El text splitting divide textos en'),
 Document(metadata={}, page_content='segmentos pequeños para facilitar s'),
 Document(metadata={}, page_content='u procesamiento. Es clave mantener'),
 Document(metadata={}, page_content='el equilibrio entre tamaño y contex'),
 Document(metadata={}, page_content='to. Un buen splitting mejora la pre'),
 Document(metadata={}, page_content='cisión en sistemas como RAG.')]

Nos devolvera una lista de _documentos_
Observa que, esta vez, tenemos los mismos chunks, pero ahora están en documentos. Estos funcionarán bien con el resto del ecosistema de LangChain. También nota que el espacio en blanco al final del tercer chunk ha desaparecido. Esto se debe a que LangChain lo elimina automáticamente. Si deseas conservar los espacios en blanco, puedes evitarlo configurando strip_whitespace=False.

**Chunk Overlap & Separadores**

El chunk overlap (solapamiento de chunks) fusiona los segmentos de texto de manera que el final del Chunk #1 coincida con el inicio del Chunk #2, y así sucesivamente. Esto ayuda a mantener el contexto entre chunks adyacentes, evitando que se pierda información importante en las divisiones.

En este caso, configuraré un solapamiento de 4 caracteres, lo que significa que los últimos 7 caracteres de un chunk serán los primeros 10 del siguiente.

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=10, separator='')

text_splitter.create_documents([text])

[Document(metadata={}, page_content='El text splitting divide textos en'),
 Document(metadata={}, page_content='vide textos en segmentos pequeños p'),
 Document(metadata={}, page_content='ntos pequeños para facilitar su pro'),
 Document(metadata={}, page_content='acilitar su procesamiento. Es clave'),
 Document(metadata={}, page_content='iento. Es clave mantener el equilib'),
 Document(metadata={}, page_content='ener el equilibrio entre tamaño y c'),
 Document(metadata={}, page_content='ntre tamaño y contexto. Un buen spl'),
 Document(metadata={}, page_content='to. Un buen splitting mejora la pre'),
 Document(metadata={}, page_content='g mejora la precisión en sistemas c'),
 Document(metadata={}, page_content='n en sistemas como RAG.')]

Una interesante herramienta para visualizar como funciona este sistema es ChunkViz.com de Gregory Kamradt




# **Nivel 2: Recursive Character Text Splitting**
---
Subamos un nivel de complejidad.

El problema con el Nivel #1 es que no tenemos en cuenta la estructura de nuestro documento en absoluto. Simplemente dividimos por un número fijo de caracteres.

El Recursive Character Text Splitter (Divisor de texto recursivo por caracteres) ayuda con esto. Con él, especificaremos una serie de separadores que se utilizarán para dividir nuestros documentos.

Puedes ver los separadores predeterminados para LangChain aquí.
https://github.com/langchain-ai/langchain/blob/9ef2feb6747f5a69d186bd623b569ad722829a5e/libs/langchain/langchain/text_splitter.py#L842

Echemos un vistazo a cada uno de ellos.

"\n\n" - Doble nueva línea, o más comúnmente saltos de párrafo.  
"\n" - Nuevas líneas.  
" " - Espacios.  
"" - Caracteres.  

### Funcionamiento Recursivo

El algoritmo se basa en una lista jerárquica de delimitadores ordenados desde el nivel más "alto" (por ejemplo, separadores de párrafo) hasta el más "bajo" (por ejemplo, espacios o caracteres individuales). El proceso es el siguiente:

1.  **Selección de Delimitadores:**
    
    -   Se define una lista de delimitadores, por ejemplo:  
        `[ "\n\n", "\n", ". ", " ", "" ]`
    -   Cada delimitador se utiliza para intentar segmentar el texto en bloques lógicos.
2.  **Fragmentación Inicial:**
    
    -   Se intenta dividir el texto usando el delimitador de mayor prioridad.
    -   Si los fragmentos resultantes cumplen con el tamaño máximo (por ejemplo, número de caracteres o tokens), el proceso se detiene.
3.  **Recursividad en Fragmentos Grandes:**
    
    -   Si algún fragmento supera el límite especificado, se aplica el mismo proceso a ese fragmento, pero usando el siguiente delimitador en la lista.
    -   Este proceso se repite recursivamente hasta que el fragmento se ajusta al límite o se llega al delimitador más bajo (que puede implicar dividir a nivel de carácter).


Probémoslo.

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = """
Uno de los aspectos más fascinantes de la IA es su capacidad para aprender y adaptarse a partir de los datos, lo que permite a los sistemas mejorar su rendimiento con el tiempo sin necesidad de programación explícita. Este enfoque, conocido como aprendizaje automático (machine learning), ha dado lugar a avances notables en áreas como el reconocimiento de voz, la traducción automática y la visión por computadora. A medida que la cantidad de datos disponibles continúa creciendo exponencialmente, el potencial de la IA para impulsar la innovación y resolver desafíos complejos parece ilimitado.

Sin embargo, el auge de la IA también plantea preguntas importantes sobre el futuro del trabajo, la ética y la privacidad. A medida que las máquinas se vuelven más inteligentes y capaces, existe la preocupación de que puedan desplazar a los trabajadores humanos en una amplia gama de industrias. Además, el uso de la IA en áreas como la vigilancia y la toma de decisiones plantea serias cuestiones éticas sobre el sesgo, la transparencia y la responsabilidad. Como dijo el renombrado físico Stephen Hawking: "La inteligencia artificial podría ser lo mejor o lo peor que le haya pasado a la humanidad". En última instancia, el futuro de la IA dependerá de cómo abordemos estos desafíos y trabajemos para garantizar que esta poderosa tecnología se utilice de manera responsable y en beneficio de toda la humanidad.
"""

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap=0)
text_splitter.create_documents([text])

[Document(metadata={}, page_content='Uno de los aspectos más fascinantes de la IA es su capacidad para aprender y adaptarse a partir de'),
 Document(metadata={}, page_content='los datos, lo que permite a los sistemas mejorar su rendimiento con el tiempo sin necesidad de'),
 Document(metadata={}, page_content='programación explícita. Este enfoque, conocido como aprendizaje automático (machine learning), ha'),
 Document(metadata={}, page_content='dado lugar a avances notables en áreas como el reconocimiento de voz, la traducción automática y la'),
 Document(metadata={}, page_content='visión por computadora. A medida que la cantidad de datos disponibles continúa creciendo'),
 Document(metadata={}, page_content='exponencialmente, el potencial de la IA para impulsar la innovación y resolver desafíos complejos'),
 Document(metadata={}, page_content='parece ilimitado.'),
 Document(metadata={}, page_content='Sin embargo, el auge de la IA también plantea preguntas importantes sobre el futuro de

### ¿Cómo se personalizan los delimitadores?
Cuando creas una instancia de la clase RecursiveCharacterTextSplitter, puedes pasarle una lista de delimitadores en el parámetro (generalmente llamado separators o similar). Esta lista define el orden y los "puntos de corte" que se utilizarán para dividir el texto.

Podria servier esto para un RAG en el que tenemos un tipo de documento muy caractreistico, por ejemplo que los documentos contuvieran textos separados por "---"

Veamos...

(Prueba diferentes configuraciones de este codigo y observa los resultados)

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter



text ="""
La inteligencia es la capacidad de adaptarse al cambio. - Stephen Hawking
====
La verdadera inteligencia no es el conocimiento, sino la imaginación. - Albert Einstein
====
"La inteligencia consiste en la capacidad de resolver problemas." - Howard Gardner
====
"La inteligencia es lo que usas cuando no sabes qué hacer." - Jean Piaget
====
"La inteligencia es la habilidad de aprender de la experiencia." - Leonardo da Vinci
====
"La inteligencia es la suma de todos tus sentidos." - Matshona Dhliwayo
====
"La inteligencia sin ambición es un pájaro sin alas." - Salvador Dalí
====
"La inteligencia es un don que te da la vida, pero depende de ti cultivarlo." - Anónimo
====
"La inteligencia no es un privilegio, es una responsabilidad." - Anónimo
====
"La inteligencia artificial nos hará cuestionar qué significa ser humano." - Yuval Noah Harari
"""

# Configuración personalizada de delimitadores
custom_separators = ["===="]
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap=0, separators=custom_separators)
chunks = text_splitter.create_documents([text])

for i,chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk} ")


Chunk 0: page_content='La inteligencia es la capacidad de adaptarse al cambio. - Stephen Hawking
====' 
Chunk 1: page_content='La verdadera inteligencia no es el conocimiento, sino la imaginación. - Albert Einstein
====' 
Chunk 2: page_content='"La inteligencia consiste en la capacidad de resolver problemas." - Howard Gardner
====' 
Chunk 3: page_content='"La inteligencia es lo que usas cuando no sabes qué hacer." - Jean Piaget
====' 
Chunk 4: page_content='"La inteligencia es la habilidad de aprender de la experiencia." - Leonardo da Vinci
====' 
Chunk 5: page_content='"La inteligencia es la suma de todos tus sentidos." - Matshona Dhliwayo
====' 
Chunk 6: page_content='"La inteligencia sin ambición es un pájaro sin alas." - Salvador Dalí
====' 
Chunk 7: page_content='"La inteligencia es un don que te da la vida, pero depende de ti cultivarlo." - Anónimo
====' 
Chunk 8: page_content='"La inteligencia no es un privilegio, es una responsabilidad." - Anónimo
====' 
Chunk 9: page_content='

Prueba a a trcear este texto

In [14]:
text = """
[frase]
**El Comienzo del Viaje**
En el corazón de un pueblo olvidado, donde las leyendas se entrelazan con la realidad, vivía Marina, una joven con una insaciable curiosidad. Desde pequeña, soñaba con explorar los confines de un mundo lleno de misterios y secretos, más allá de los límites de su tranquilo hogar.
[/frase]
[frase]**El Misterio del Bosque Encantado**
Un día, Marina encontró un antiguo mapa en el desván de su abuela. El mapa, trazado a mano y salpicado de símbolos enigmáticos, señalaba la existencia de un bosque encantado, donde se decía que la naturaleza cobraba vida y las flores susurraban secretos olvidados. Decidida a descubrir la verdad, empacó sus cosas y partió hacia la aventura.
[/frase]
[frase]**Encuentros Sobrenaturales**
A medida que avanzaba, el bosque se volvía cada vez más denso y misterioso. Entre los árboles centenarios, Marina se encontró con criaturas singulares: un zorro de pelaje plateado que parecía conocer todos los senderos, y un anciano roble que, según contaban, había sido testigo de los orígenes del mundo. Cada encuentro la llenaba de asombro y la impulsaba a seguir adelante, guiada por la esperanza y el eco de viejas leyendas.
[/frase]
[frase]**El Secreto Revelado**
Finalmente, tras superar desafíos que parecían sacados de un sueño, Marina llegó a un claro iluminado por una luz dorada. Allí, en medio de un círculo de antiguas piedras, descubrió un relicario custodiado por una energía ancestral. El relicario contenía la clave para restaurar el equilibrio perdido entre la magia y la realidad, un legado destinado a cambiar el destino de su pueblo para siempre.
[/frase]
[frase]**El Regreso Transformado**
Con el relicario en sus manos y el conocimiento adquirido en su travesía, Marina regresó a su hogar. La joven, ahora transformada por la experiencia, se convirtió en la guardiana de una nueva era, donde las fronteras entre lo cotidiano y lo extraordinario se difuminaban. Su historia inspiró a generaciones enteras a buscar la magia oculta en cada rincón del mundo.
[/frase]
"""


from langchain.text_splitter import RecursiveCharacterTextSplitter

custom_separators = ["[frase]"]  # Puedes agregar más niveles si lo necesitas

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,          # Tamaño máximo del fragmento (ajustable)
    chunk_overlap=50,        # Solapamiento entre fragmentos
    separators=custom_separators
)


chunks = text_splitter.split_text(text)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} ------ :\n{chunk}\n")


Chunk 1 ------ :
[frase]
**El Comienzo del Viaje**
En el corazón de un pueblo olvidado, donde las leyendas se entrelazan con la realidad, vivía Marina, una joven con una insaciable curiosidad. Desde pequeña, soñaba con explorar los confines de un mundo lleno de misterios y secretos, más allá de los límites de su tranquilo hogar.
[/frase]

Chunk 2 ------ :
[frase]**El Misterio del Bosque Encantado**
Un día, Marina encontró un antiguo mapa en el desván de su abuela. El mapa, trazado a mano y salpicado de símbolos enigmáticos, señalaba la existencia de un bosque encantado, donde se decía que la naturaleza cobraba vida y las flores susurraban secretos olvidados. Decidida a descubrir la verdad, empacó sus cosas y partió hacia la aventura.
[/frase]

Chunk 3 ------ :
[frase]**Encuentros Sobrenaturales**
A medida que avanzaba, el bosque se volvía cada vez más denso y misterioso. Entre los árboles centenarios, Marina se encontró con criaturas singulares: un zorro de pelaje plateado que parecía 

# **Nivel 3: División Específica de Documentos**

Subiendo por nuestra escalera de niveles, comencemos a manejar tipos de documentos que no sean prosa normal en un archivo .txt. ¿Qué pasa si tienes imágenes? ¿O un PDF? ¿O fragmentos de código?

Nuestros dos primeros niveles no funcionarían muy bien para esto, por lo que necesitaremos encontrar una táctica diferente.

Este nivel se trata de hacer que tu estrategia de troceado se ajuste a tus diferentes formatos de datos. Repasemos varios ejemplos de esto en acción.

Los divisores de Markdown, Python y JS serán básicamente similares a Recursive Character, pero con diferentes separadores.

### Markdown

Puedes ver los divisores aqui: https://github.com/langchain-ai/langchain/blob/9ef2feb6747f5a69d186bd623b569ad722829a5e/libs/langchain/langchain/text_splitter.py#L1175

Separators:
```
\n#{1,6} - Split by new lines followed by a header (H1 through H6)
```\n - Code blocks
\n\\*\\*\\*+\n - Horizontal Lines
\n---+\n - Horizontal Lines
\n___+\n - Horizontal Lines
\n\n Double new lines
\n - New line
" " - Spaces
"" - Characte


In [6]:
from langchain.text_splitter import MarkdownTextSplitter

splitter = MarkdownTextSplitter(chunk_size = 600, chunk_overlap=0)

text = """
# **Text Splitting**
---
El **Text Splitting** (o división de texto) es una etapa crítica en los sistemas de **Retrieval-Augmented Generation (RAG)**. Consiste en fragmentar documentos extensos en segmentos más pequeños y manejables (_chunks_) antes de indexarlos para su recuperación posterior. Su implementación incide directamente en la calidad de la recuperación de información y, por ende, en las respuestas generadas por el modelo.

---

### **¿Por qué es importante?**

1.  **Equilibrio entre contexto y precisión**:

    -   **Chunks demasiado grandes**: Pueden contener información redundante o irrelevante, lo que "diluye" el contexto clave y reduce la precisión del retrieval.

    -   **Chunks demasiado pequeños**: Pierden contexto necesario para entender el significado completo (p. ej., una frase sin su párrafo asociado).

    -   Un buen _splitting_ mantiene la coherencia semántica en cada segmento, facilitando que el sistema recupere los fragmentos más relevantes para una consulta.

2.  **Compatibilidad con modelos de embedding**:

    -   Los modelos de embedding (como SBERT o OpenAI) tienen límites óptimos de longitud de texto. Por ejemplo, un chunk de 512 tokens funciona bien con muchos codificadores, pero un texto más largo podría truncarse o perder información crítica.

"""

chunks = splitter.create_documents([text])
for chunk in chunks:
    print(chunk)
    print("-----")


page_content='# **Text Splitting**
---
El **Text Splitting** (o división de texto) es una etapa crítica en los sistemas de **Retrieval-Augmented Generation (RAG)**. Consiste en fragmentar documentos extensos en segmentos más pequeños y manejables (_chunks_) antes de indexarlos para su recuperación posterior. Su implementación incide directamente en la calidad de la recuperación de información y, por ende, en las respuestas generadas por el modelo.  

---

### **¿Por qué es importante?**'
-----
page_content='1.  **Equilibrio entre contexto y precisión**:
    
    -   **Chunks demasiado grandes**: Pueden contener información redundante o irrelevante, lo que "diluye" el contexto clave y reduce la precisión del retrieval.
        
    -   **Chunks demasiado pequeños**: Pierden contexto necesario para entender el significado completo (p. ej., una frase sin su párrafo asociado).
        
    -   Un buen _splitting_ mantiene la coherencia semántica en cada segmento, facilitando que el sistema